# **Tutorial: Guía Interactiva de Qwen3-TTS**

**Autor:** Gemini CLI
**Modelo:** Qwen3-TTS (Versiones 1.7B y 0.6B)
**Tarea:** Texto a Voz (TTS), Clonación de Voz, Diseño de Voz

Este notebook demuestra las capacidades de **Qwen3-TTS**, un modelo de generación de voz de última generación. Exploraremos:
1.  **Voz Personalizada (Custom Voice):** Generación de voz utilizando personajes preestablecidos de alta calidad.
2.  **Diseño de Voz (Voice Design):** Creación de voces totalmente nuevas utilizando descripciones en lenguaje natural.
3.  **Clonación de Voz (Voice Cloning):** Clonación de una voz a partir de un clip de audio de referencia corto (Zero-Shot).

---

![image.png](attachment:image.png)

## **Paso 1: Instalación y Configuración**

Primero, necesitamos instalar el paquete `qwen-tts`. También instalaremos `flash-attn` para una generación más rápida, aunque requiere una GPU (se recomienda T4 o superior).

*Nota: La compilación de `flash-attn` puede tardar unos minutos. Si tienes prisa o estás en una CPU, puedes omitirlo, pero debes eliminar `attn_implementation="flash_attention_2"` del código de carga del modelo posterior.*

In [3]:
# @title Instalar Dependencias
!pip install -U qwen-tts
!pip install -U flash-attn --no-build-isolation
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00
  Created wheel for sox: filename=sox-1.5.0-py3-none-any.whl size=40036 sha256=99c44420aaee0d535517944d50714008284a4413ea3ac4ea8ce6593a5a4bea91
  Stored in directory: /root/.cache/pip/wheels/8c/c7/e7/baea1f7e79b9eb53addc81cc9b827424f4a7d8c9cc18c03659
Successfully built sox
  Attempting uninstall: transformers
    Found exi

In [5]:
# @title Importar Librerías y Verificar GPU
import torch
import soundfile as sf
from IPython.display import Audio, display
import os

# Verificar si CUDA (GPU) está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Advertencia: Ejecutando en CPU. La generación será lenta.")

# Función auxiliar para reproducir audio directamente en el notebook
def play_audio(file_path):
    if os.path.exists(file_path):
        display(Audio(file_path))
    else:
        print(f"Archivo no encontrado: {file_path}")

Usando dispositivo: cpu
Advertencia: Ejecutando en CPU. La generación será lenta.


---

## **Paso 2: Generación de Voz Personalizada (Personajes Preestablecidos)**

Qwen3-TTS viene con varias voces preestablecidas de alta calidad (por ejemplo, Vivian, Ryan, Tanaka). Este modo es mejor para texto a voz general donde deseas un personaje específico y estable.

### **Cargar Modelo**
Usaremos el modelo `Qwen3-TTS-12Hz-1.7B-CustomVoice`.

In [6]:
from qwen_tts import Qwen3TTSModel

# @title Seleccionar y Cargar Modelo
model_name = "Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice" # @param ["Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice", "Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice"]

print(f"Cargando {model_name}...")

# Cargar el modelo Custom Voice
# Nota: Si no instalaste flash-attn, elimina el argumento `attn_implementation`.
custom_voice_model = Qwen3TTSModel.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    attn_implementation="flash_attention_2" if device == "cuda" else None,
)

print("¡Modelo de Voz Personalizada Cargado!")


    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    



********
********
 
Cargando Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

¡Modelo de Voz Personalizada Cargado!


### **Generar Voz**
Hagamos que "Vivian" (Chino) o "Ryan" (Inglés) hablen. También puedes proporcionar una instrucción (`instruct`) para modificar la emoción.

![image.png](attachment:image.png)

In [7]:
# @title Generar Voces Preestablecidas

text_input = "¡Hola! Esta es una demostración de Qwen3-TTS. Puedo hablar con emociones claras."
speaker_name = "Sohee" # @param ["Vivian", "Serena", "Uncle_Fu", "Dylan", "Eric", "Ryan", "Aiden", "Ono_Anna", "Sohee"]
instruction = "Feliz y enérgica"

print(f"Generando audio para {speaker_name}...")

wavs, sr = custom_voice_model.generate_custom_voice(
    text=text_input,
    language="Auto",  # Detecta automáticamente el idioma
    speaker=speaker_name,
    instruct=instruction
)

output_path = f"output_{speaker_name}.wav"
sf.write(output_path, wavs[0], sr)

print(f"Guardado en {output_path}")
play_audio(output_path)

Generando audio para Sohee...


Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


Guardado en output_Sohee.wav


---

## **Paso 3: Diseño de Voz (Texto a Voz)**

Esta función te permite **describir** una voz utilizando lenguaje natural, y el modelo generará un habla que coincida con esa descripción. Esto es poderoso para crear NPCs o personajes específicos sin audio de referencia.

### **Cargar Modelo**
Necesitamos cargar una variante específica del modelo para esto: `Qwen3-TTS-12Hz-1.7B-VoiceDesign`.

In [16]:
# Limpiar modelo anterior para ahorrar VRAM
del custom_voice_model
torch.cuda.empty_cache()

# Cargar el modelo Voice Design
voice_design_model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign",
    device_map="auto",
    dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    attn_implementation="flash_attention_2" if device == "cuda" else None,
)

print("¡Modelo de Diseño de Voz Cargado!")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

¡Modelo de Diseño de Voz Cargado!


### **Diseñar una Voz**
Intenta describir edad, género, tono, velocidad de habla y emoción.

In [15]:
text_to_speak = "No puedo creer que finalmente llegamos a la cima de la montaña. ¡La vista es increíble!"
# Descripción: Un hombre de mediana edad con voz grave y profunda, hablando lentamente y sonando exhausto pero asombrado.
voice_description = '''gender: Male
pitch: Deep and resonant with subtle downward inflections suggesting gravity
speed: Deliberately slow with extended pauses between sentences
volume: Moderate to soft, creating an intimate atmosphere
age: Middle-aged to older adult
clarity: Crystal clear enunciation with careful articulation
fluency: Smooth and controlled with intentional dramatic pauses
accent: Standard American English
texture: Rich and velvety with a slightly smoky quality
emotion: Contemplative and intriguing
tone: Mysterious, philosophical, and atmospheric
personality: Introspective, wise, and captivating '''

print("Diseñando voz y generando...")

# Nota: Las instrucciones de diseño suelen funcionar mejor en Inglés, aunque el texto a hablar sea español.
wavs, sr = voice_design_model.generate_voice_design(
    text=text_to_speak,
    language="Spanish", # Especificamos el idioma del texto
    instruct=voice_description
)

output_design_path = "output_voice_design.wav"
sf.write(output_design_path, wavs[0], sr)

play_audio(output_design_path)

Diseñando voz y generando...


NameError: name 'voice_design_model' is not defined

In [13]:
text_to_speak = "No puedo creer que finalmente llegamos a la cima de la montaña. ¡La vista es increíble!"
# Descripción: Un hombre de mediana edad con voz grave y profunda, hablando lentamente y sonando exhausto pero asombrado.
voice_description = '''gender: Female
pitch: Medium-low female pitch with gentle, soothing fluctuations
speed: Very slow and measured, allowing time for mental processing
volume: Soft and calming, never raising above comfortable levels
age: Adult (30s-40s)
clarity: Exceptionally clear with soft consonants
fluency: Perfectly fluid with mindful breathing pauses
accent: Neutral North American with slight California influence
texture: Warm and breathy, incredibly smooth
emotion: Peaceful and nurturing
tone: Gentle, encouraging, and meditative
personality: Compassionate, patient, and serene'''
print("Diseñando voz y generando...")

# Nota: Las instrucciones de diseño suelen funcionar mejor en Inglés, aunque el texto a hablar sea español.
wavs, sr = voice_design_model.generate_voice_design(
    text=text_to_speak,
    language="Spanish", # Especificamos el idioma del texto
    instruct=voice_description
)

output_design_path = "output_voice_design.wav"
sf.write(output_design_path, wavs[0], sr)

play_audio(output_design_path)

Diseñando voz y generando...


NameError: name 'voice_design_model' is not defined

In [12]:
text_to_speak = "No puedo creer que finalmente llegamos a la cima de la montaña. ¡La vista es increíble!"
# Descripción: Un hombre de mediana edad con voz grave y profunda, hablando lentamente y sonando exhausto pero asombrado.
voice_description ='''gender: Male
pitch: High child's voice with wide pitch variations for storytelling
speed: Variable - rushing through exciting parts, slowing for details
volume: Moderate with sudden louder bursts during exciting moments
age: Child (8-10 years old)
clarity: Generally clear but with occasional word stumbles
fluency: Enthusiastic flow with natural childlike interruptions
accent: American English (General American)
texture: Bright and youthful with slight breathiness
emotion: Wonder and excitement mixed with nervousness
tone: Animated, imaginative, and earnest
personality: Innocent, creative, and eager to share'''

print("Diseñando voz y generando...")

# Nota: Las instrucciones de diseño suelen funcionar mejor en Inglés, aunque el texto a hablar sea español.
wavs, sr = voice_design_model.generate_voice_design(
    text=text_to_speak,
    language="Spanish", # Especificamos el idioma del texto
    instruct=voice_description
)

output_design_path = "output_voice_design.wav"
sf.write(output_design_path, wavs[0], sr)

play_audio(output_design_path)

Diseñando voz y generando...


NameError: name 'voice_design_model' is not defined

---

## **Paso 4: Clonación de Voz (Zero-Shot)**

El modelo "Base" te permite clonar cualquier voz utilizando un clip de audio de referencia corto (3-10 segundos).

### **Cargar Modelo**
Usamos `Qwen3-TTS-12Hz-1.7B-Base`.

In [17]:
# Limpiar memoria
del voice_design_model
torch.cuda.empty_cache()

# @title Seleccionar y Cargar Modelo Base
model_name = "Qwen/Qwen3-TTS-12Hz-1.7B-Base" # @param ["Qwen/Qwen3-TTS-12Hz-1.7B-Base", "Qwen/Qwen3-TTS-12Hz-0.6B-Base"]

print(f"Cargando {model_name}...")

# Cargar el modelo Base/Clone
base_model = Qwen3TTSModel.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    attn_implementation="flash_attention_2" if device == "cuda" else None,
)

print("¡Modelo de Clonación de Voz Cargado!")

Cargando Qwen/Qwen3-TTS-12Hz-1.7B-Base...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

¡Modelo de Clonación de Voz Cargado!


### **Clonar una Voz**
Puedes usar una URL a un archivo de audio o subir uno a Colab. Aquí usamos una URL de ejemplo del repositorio.

In [18]:
# Puedes reemplazar esta URL con una ruta a un archivo local (ej. "/content/mi_voz.wav")
ref_audio_path = "https://drive.google.com/uc?export=download&id=1QdauC5wePs7YgxUaZ9U5mctEtmQRgRaG"
#ref_audio_path = "https://drive.google.com/uc?export=download&id=1dYl1634xT4UBAclsNboRMCbq-7EWTOj_"
ref_audio_text = "Karol y Tim están llegando al destino de sus sueños. Tienen tras de sí un viaje de más de 10 horas en coche, del frío sur al trópico más caluroso. Están a punto de descubrir la extraordinaria Gran Barrera de Coral. Con sus 2300 kilómetros de largo y 300 arrecifes de coral individuales, se encuentran muy cerca del arrecife de coral más grande del mundo. Karol y Tim se ponen sus trajes de buceo, les espera una hora de recorrido en barco."

print(f"Clonando voz desde referencia...")

# 1. Crear estructura del prompt (opcional pero recomendado para reusar)
# Esto extrae las características de estilo del audio de referencia
voice_clone_prompt = base_model.create_voice_clone_prompt(
    ref_audio=ref_audio_path,
    ref_text=ref_audio_text
)

Clonando voz desde referencia...


### **Haciendo inferencia de la voz clonada**

In [19]:
# 2. Generar - Spanish example
target_text = "El departamento de informática, no esta disponible ahora. Porfavor, intentelo de nuevo mas tarde o deje un correo electronico. Gracias."

wavs, sr = base_model.generate_voice_clone(
    text=target_text,
    language="Spanish",
    voice_clone_prompt=voice_clone_prompt
)

output_clone_path = "output_cloned.wav"
sf.write(output_clone_path, wavs[0], sr)

print("Referencia Original:")
# Si es una URL, no podemos reproducirla directamente con IPython.display(Audio(url)) fácilmente en todos los navegadores sin descargar,
# pero para archivos locales funciona.
print(f"(Fuente de referencia: {ref_audio_path})")

print("\nResultado Clonado:")
play_audio(output_clone_path)

Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


Referencia Original:
(Fuente de referencia: https://drive.google.com/uc?export=download&id=1QdauC5wePs7YgxUaZ9U5mctEtmQRgRaG)

Resultado Clonado:


In [34]:
# 2. Generar - English example
target_text = "This is another example of voice cloning. The resemblance is quite amazing, isn't it?"

wavs, sr = base_model.generate_voice_clone(
    text=target_text,
    language="English",
    voice_clone_prompt=voice_clone_prompt
)

output_clone_path = "output_cloned.wav"
sf.write(output_clone_path, wavs[0], sr)

print("Referencia Original:")
# Si es una URL, no podemos reproducirla directamente con IPython.display(Audio(url)) fácilmente en todos los navegadores sin descargar,
# pero para archivos locales funciona.
print(f"(Fuente de referencia: {ref_audio_path})")

print("\nResultado Clonado:")
play_audio(output_clone_path)

Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


Referencia Original:
(Fuente de referencia: https://drive.google.com/uc?export=download&id=1QdauC5wePs7YgxUaZ9U5mctEtmQRgRaG)

Resultado Clonado:


---

## **Paso 5: Modelos Ligeros (0.6B)**

Los modelos de 0.6B son significativamente más rápidos y requieren menos VRAM. Son ideales para prototipado rápido o hardware limitado.

Primero, **limpiaremos la memoria de la GPU** para asegurarnos de tener espacio suficiente.

In [ ]:
# Limpiar memoria de la GPU
try:
    del base_model
except NameError:
    pass
try:
    del custom_voice_model
except NameError:
    pass
try:
    del voice_design_model
except NameError:
    pass

import gc
gc.collect()
torch.cuda.empty_cache()
print("Memoria GPU liberada.")

In [ ]:
# Cargar modelo 0.6B CustomVoice
from qwen_tts import Qwen3TTSModel

print("Cargando Qwen3-TTS-12Hz-0.6B-CustomVoice...")
model_06b = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice",
    device_map="auto",
    dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    attn_implementation="flash_attention_2" if device == "cuda" else None,
)

print("Modelo 0.6B Cargado. Probando generación...")

wavs, sr = model_06b.generate_custom_voice(
    text="¡Hola! Soy la versión 0.6B, soy más rápida y ligera.",
    language="Spanish",
    speaker="Ryan"
)

sf.write("output_06b.wav", wavs[0], sr)
play_audio("output_06b.wav")

---

## **Resumen**

En este notebook, hemos cubierto:
1.  **CustomVoice:** Usando `generate_custom_voice` con presets como "Ryan" o "Vivian".
2.  **VoiceDesign:** Usando `generate_voice_design` para crear voces a partir de instrucciones de texto (ej. "Deep, scary monster voice").
3.  **VoiceClone:** Usando `generate_voice_clone` con el modelo Base para copiar una voz desde un archivo de referencia.

**Consejos para Mejores Resultados:**
*   **Ingeniería de Prompts:** Para el Diseño de Voz, sé descriptivo. Menciona género, edad, tono, velocidad y emoción (preferiblemente en inglés para el prompt de descripción).
*   **Audio de Referencia:** Para Clonación, usa audio limpio sin música de fondo. 3-10 segundos suelen ser suficientes.
*   **Idioma:** El modelo soporta 10 idiomas (Inglés, Chino, Japonés, Coreano, Alemán, Francés, Ruso, Portugués, Español, Italiano). Puedes cambiar el parámetro `language` según corresponda.